<a href="https://colab.research.google.com/github/rahulombale/swing_strategy_rahul/blob/main/Moving_Average_Contrarian_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import plotly.graph_objects as go
from google.colab import drive
from datetime import datetime
import os

# --- 1. SETUP AND CONFIGURATION ---

# Mount Google Drive
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"Could not mount drive: {e}")

# --- INPUT VARIABLES ---
DATA_FOLDER_DATE = '2025-06-17'
DATA_BASE_PATH = '/content/drive/My Drive/stock_data/'
BASE_DATA_PATH = f'{DATA_BASE_PATH}{DATA_FOLDER_DATE}/'
SOURCE_FILE_V40 = 'v40_token.csv' # This strategy is only for V40 stocks

# --- OUTPUT FILE NAMES ---
COMPLETED_TRADES_FILE = 'ma_strategy_completed.csv'
OPEN_TRADES_FILE = 'ma_strategy_open.csv'

# Strategy Parameters
CAPITAL_PER_TRADE = 100000  # Note: The prompt mentioned "3% in one trade". This is interpreted as a fixed capital amount for simplicity.
AVERAGE_DOWN_PERCENT = -0.10 # -10% drop to trigger averaging
TICKER_COLUMN_NAME = 'ticker'


# --- 2. CORE FUNCTIONS ---

def load_ma_data(file_path, stock_name_for_print):
    """Loads and preprocesses stock data, calculating multiple SMAs."""
    if not os.path.exists(file_path):
        print(f"      - WARNING: Data file not found for {stock_name_for_print} at {file_path}. Skipping.")
        return None
    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        df.sort_index(inplace=True)
        # Calculate all required SMAs
        df['sma_20'] = df['close'].rolling(window=20).mean()
        df['sma_50'] = df['close'].rolling(window=50).mean()
        df['sma_200'] = df['close'].rolling(window=200).mean()
        # Drop rows with NaN values that result from SMA calculation
        df.dropna(inplace=True)
        return df
    except Exception as e:
        print(f"      - ERROR: Could not process file for {stock_name_for_print}. Reason: {e}")
        return None

def execute_ma_backtest(df, stock_name, capital):
    """
    Executes the Moving Average Contrarian Strategy backtest for a single stock.
    """
    completed_trades = []
    open_trades = []

    in_position = False
    position_details = {} # To store info like entry_price, shares, average_count etc.

    # We start from the second row to use the previous day's close for decisions
    for i in range(1, len(df)):
        prev_row = df.iloc[i-1]
        curr_row = df.iloc[i]

        # --- A. CHECK FOR TRADE ACTION ---
        if not in_position:
            # --- CHECK FOR BUY SIGNAL ---
            buy_condition = (prev_row['close'] < prev_row['sma_20'] and
                             prev_row['sma_20'] < prev_row['sma_50'] and
                             prev_row['sma_50'] < prev_row['sma_200'])

            if buy_condition:
                entry_price = curr_row['open']
                num_shares = int(capital / entry_price)
                if num_shares == 0: continue

                in_position = True
                position_details = {
                    'entry_date': curr_row.name,
                    'initial_entry_price': entry_price,
                    'avg_price': entry_price,
                    'total_shares': num_shares,
                    'total_investment': num_shares * entry_price,
                    'average_count': 0,
                    'buy_points': [(curr_row.name, entry_price)] # List to store buy dates and prices
                }
                print(f"  - [{stock_name} @ {curr_row.name.date()}] BUY SIGNAL TRIGGERED. Buying {num_shares} shares at {entry_price:.2f}")

        else: # If we are already in a position
            # --- 1. CHECK FOR AVERAGING DOWN ---
            price_change_from_initial = (curr_row['open'] - position_details['initial_entry_price']) / position_details['initial_entry_price']

            if position_details['average_count'] < 1 and price_change_from_initial <= AVERAGE_DOWN_PERCENT:
                avg_price = curr_row['open']
                num_shares = int(capital / avg_price)
                if num_shares == 0: continue

                # Update position details after averaging
                new_investment = num_shares * avg_price
                position_details['total_investment'] += new_investment
                position_details['total_shares'] += num_shares
                position_details['avg_price'] = position_details['total_investment'] / position_details['total_shares']
                position_details['average_count'] += 1
                position_details['buy_points'].append((curr_row.name, avg_price))
                print(f"  - [{stock_name} @ {curr_row.name.date()}] AVERAGING DOWN. Buying {num_shares} more shares at {avg_price:.2f}")

            # --- 2. CHECK FOR SELL SIGNAL ---
            sell_condition = (prev_row['close'] > prev_row['sma_20'] and
                              prev_row['sma_20'] > prev_row['sma_50'] and
                              prev_row['sma_50'] > prev_row['sma_200'])

            if sell_condition:
                exit_price = curr_row['open']
                sale_value = position_details['total_shares'] * exit_price
                profit = sale_value - position_details['total_investment']

                completed_trades.append({
                    'Stock': stock_name,
                    'Entry Date': position_details['entry_date'],
                    'Exit Date': curr_row.name,
                    'Avg Entry Price': position_details['avg_price'],
                    'Exit Price': exit_price,
                    'Investment': position_details['total_investment'],
                    'Sale Value': sale_value,
                    'Profit': profit,
                    'Profit %': (profit / position_details['total_investment']) * 100,
                    'Times Averaged': position_details['average_count'],
                    'Buy Points': position_details['buy_points']
                })
                print(f"  - [{stock_name} @ {curr_row.name.date()}] SELL SIGNAL TRIGGERED. Selling {position_details['total_shares']} shares at {exit_price:.2f}")

                # Reset state
                in_position = False
                position_details = {}

    # --- B. HANDLE OPEN TRADES AT THE END OF DATA ---
    if in_position:
        last_close = df.iloc[-1]['close']
        unrealized_pnl = (last_close * position_details['total_shares']) - position_details['total_investment']

        open_trades.append({
            'Stock': stock_name,
            'Status': 'Open',
            'Entry Date': position_details['entry_date'],
            'Avg Entry Price': position_details['avg_price'],
            'Investment': position_details['total_investment'],
            'Last Known Date': df.index[-1],
            'Last Price': last_close,
            'Unrealized P/L': unrealized_pnl,
            'Unrealized P/L %': (unrealized_pnl / position_details['total_investment']) * 100,
            'Shares': position_details['total_shares']
        })

    return completed_trades, open_trades


def plot_ma_trade(df, trade):
    """Generates an interactive chart for a completed MA strategy trade."""
    plot_start = trade['Entry Date'] - pd.Timedelta(days=30)
    plot_end = trade['Exit Date'] + pd.Timedelta(days=30)
    chart_df = df.loc[plot_start:plot_end]

    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(x=chart_df.index, open=chart_df['open'], high=chart_df['high'],
                                  low=chart_df['low'], close=chart_df['close'], name=trade['Stock']))
    # SMA Lines
    fig.add_trace(go.Scatter(x=chart_df.index, y=chart_df['sma_20'], mode='lines', name='20 SMA', line=dict(color='green', width=1)))
    fig.add_trace(go.Scatter(x=chart_df.index, y=chart_df['sma_50'], mode='lines', name='50 SMA', line=dict(color='red', width=1)))
    fig.add_trace(go.Scatter(x=chart_df.index, y=chart_df['sma_200'], mode='lines', name='200 SMA', line=dict(color='black', width=1.5)))

    # Mark Buy and Sell points
    buy_dates = [p[0] for p in trade['Buy Points']]
    buy_prices = [p[1] for p in trade['Buy Points']]
    fig.add_trace(go.Scatter(x=buy_dates, y=buy_prices, mode='markers', name='Buy Points',
                              marker=dict(color='blue', size=12, symbol='triangle-up', line=dict(width=2, color='white'))))
    fig.add_trace(go.Scatter(x=[trade['Exit Date']], y=[trade['Exit Price']], mode='markers', name='Sell Point',
                              marker=dict(color='fuchsia', size=12, symbol='triangle-down', line=dict(width=2, color='white'))))

    fig.update_layout(
        title=f"Moving Average Trade: {trade['Stock']} ({trade['Entry Date'].date()} to {trade['Exit Date'].date()})",
        xaxis_title="Date", yaxis_title="Price (INR)", xaxis_rangeslider_visible=False,
        template="plotly_white", legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
    fig.show()


# --- 3. MAIN EXECUTION BLOCK with Toggle ---
if __name__ == '__main__':
    # --- TOGGLE AND SETTINGS ---
    RUN_SINGLE_STOCK = False
    TICKER_TO_TEST = 'ASIANPAINT-EQ'
    # ---------------------------

    all_completed_trades = []
    all_open_trades = []

    if RUN_SINGLE_STOCK:
        print(f"--- Running single stock analysis for: {TICKER_TO_TEST} ---")
        data_file_path = f"{BASE_DATA_PATH}{TICKER_TO_TEST}.csv"
        df = load_ma_data(data_file_path, TICKER_TO_TEST)
        if df is not None:
            completed, opened = execute_ma_backtest(df, TICKER_TO_TEST, CAPITAL_PER_TRADE)
            if completed:
                all_completed_trades.extend(completed)
                for trade in completed:
                    plot_ma_trade(df, trade)
            if opened:
                all_open_trades.extend(opened)
    else:
        print("--- Starting Moving Average Strategy Batch Backtest (V40 Stocks Only) ---")
        stock_list_path = os.path.join(DATA_BASE_PATH, SOURCE_FILE_V40)
        try:
            stocks_df = pd.read_csv(stock_list_path)
            tickers = stocks_df[TICKER_COLUMN_NAME].dropna().unique()
            print(f"  - Found {len(tickers)} unique stocks in {SOURCE_FILE_V40}.")
            for ticker in tickers:
                print(f"\n[2] Analyzing stock: {ticker}")
                data_file_path = f"{BASE_DATA_PATH}{ticker}.csv"
                df = load_ma_data(data_file_path, ticker)
                if df is not None:
                    completed, opened = execute_ma_backtest(df, ticker, CAPITAL_PER_TRADE)
                    if completed:
                        all_completed_trades.extend(completed)
                    if opened:
                        all_open_trades.extend(opened)
        except FileNotFoundError:
            print(f"  - ERROR: Stock list file not found at {stock_list_path}.")

    # --- 4. FINAL SUMMARY AND OUTPUT ---
    print("\n" + "="*82)
    print("--- BACKTESTING COMPLETE: FINAL SUMMARY ---")
    print("="*82)

    if all_completed_trades:
        completed_df = pd.DataFrame(all_completed_trades).drop(columns=['Buy Points']) # Drop list column for clean CSV
        print("\n" + "="*28 + " COMPLETED TRADES SUMMARY " + "="*28)
        print(completed_df.to_string())
        output_path = os.path.join(DATA_BASE_PATH, COMPLETED_TRADES_FILE)
        completed_df.to_csv(output_path, index=False)
        print(f"\nSaved completed trades summary to '{output_path}'")
    else:
        print("\nNo completed trades were found.")

    if all_open_trades:
        open_df = pd.DataFrame(all_open_trades)
        print("\n" + "="*24 + " OPEN TRADES (TARGET NOT MET) SUMMARY " + "="*23)
        print(open_df.to_string())
        output_path = os.path.join(DATA_BASE_PATH, OPEN_TRADES_FILE)
        open_df.to_csv(output_path, index=False)
        print(f"\nSaved open trades summary to '{output_path}'")
    else:
        print("\nNo open trades were found at the end of the backtest period.")

Mounted at /content/drive
--- Starting Moving Average Strategy Batch Backtest (V40 Stocks Only) ---
  - Found 40 unique stocks in v40_token.csv.

[2] Analyzing stock: LT-EQ
  - [LT-EQ @ 2020-10-08] BUY SIGNAL TRIGGERED. Buying 112 shares at 890.30
  - [LT-EQ @ 2020-11-27] SELL SIGNAL TRIGGERED. Selling 112 shares at 1129.00
  - [LT-EQ @ 2022-04-27] BUY SIGNAL TRIGGERED. Buying 59 shares at 1675.00
  - [LT-EQ @ 2022-06-20] AVERAGING DOWN. Buying 67 more shares at 1490.00
  - [LT-EQ @ 2022-09-08] SELL SIGNAL TRIGGERED. Selling 126 shares at 1967.90
  - [LT-EQ @ 2025-02-13] BUY SIGNAL TRIGGERED. Buying 30 shares at 3289.65

[2] Analyzing stock: RELIANCE-EQ
  - [RELIANCE-EQ @ 2021-04-13] BUY SIGNAL TRIGGERED. Buying 114 shares at 873.36
  - [RELIANCE-EQ @ 2021-08-27] SELL SIGNAL TRIGGERED. Selling 114 shares at 1015.44
  - [RELIANCE-EQ @ 2023-02-03] BUY SIGNAL TRIGGERED. Buying 93 shares at 1066.28
  - [RELIANCE-EQ @ 2023-06-30] SELL SIGNAL TRIGGERED. Selling 93 shares at 1155.89
  - [RELI